In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [2]:
# !pip install chart_studio

import re
# Tutorial about Python regular expressions: https://pymotw.com/2/re/ import string
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
import nltk
import spacy
import random
from spacy.util import compounding
from spacy.util import minibatch

import warnings
warnings.filterwarnings("ignore")

In [4]:
BASE_PATH = 'ModelData/'

train_df = pd.read_csv('SkillSets_eng_all.csv', encoding= 'unicode_escape')
train_df

,id,text,selected_text
0,1,just ran thr contract negotiation ough the rain,contract negotiation
1,2,Another loser: ****. I have lost the game more...,supply chain management
2,3,"Went to get dog from vets, theyve stitched her...",cost reduction
3,4,I made my parents add u guys on the family st...,strategic sourcing
4,5,Link di materials dn`t work,materials
...,...,...,...
16344443,16344433,I`ve listed lots of enterprise software new b...,enterprise software
16344444,16344434,I`m not value based selling a hobo.,value based selling
16344445,16344435,That would have been fun BUT we were soooo TI...,sales presentations
16344446,16344436,"here btw, thi selling s is not a bot",selling


In [5]:
train_df = train_df.dropna()
train_df

,id,text,selected_text
0,1,just ran thr contract negotiation ough the rain,contract negotiation
1,2,Another loser: ****. I have lost the game more...,supply chain management
2,3,"Went to get dog from vets, theyve stitched her...",cost reduction
3,4,I made my parents add u guys on the family st...,strategic sourcing
4,5,Link di materials dn`t work,materials
...,...,...,...
16344443,16344433,I`ve listed lots of enterprise software new b...,enterprise software
16344444,16344434,I`m not value based selling a hobo.,value based selling
16344445,16344435,That would have been fun BUT we were soooo TI...,sales presentations
16344446,16344436,"here btw, thi selling s is not a bot",selling


In [6]:
print(f'{BASE_PATH}/SkillModel')

ModelData//SkillModel


In [21]:
def save_model(output_dir, nlp, new_model_name):
    output_dir = f'{BASE_PATH}/SkillModel'
    if output_dir is not None:        
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        nlp.meta["name"] = new_model_name
        nlp.to_disk(output_dir)
        print("Saved model to", output_dir)

In [8]:
# pass model = nlp if you want to train on top of existing model 

def train(train_data, output_dir, n_iter=20, model=None):
    """Load the model, set up the pipeline and train the entity recognizer."""
    ""
    if model is not None:
        nlp = spacy.load(output_dir)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")
    
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")
    
    # add labels
    for _, annotations in train_data:
        for ent in annotations.get("entities"):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != "ner"]
    with nlp.disable_pipes(*other_pipes):  # only train NER
        # sizes = compounding(1.0, 4.0, 1.001)
        # batch up the examples using spaCy's minibatch
#         if model is None:
#             nlp.begin_training()
#         else:
#             nlp.resume_training()
        optimizer = nlp.begin_training()

        for itn in tqdm(range(n_iter)):
            random.shuffle(train_data)
#             batches = minibatch(train_data, size=compounding(4.0, 500.0, 1.001))    
            losses = {}
            for text, annotations in train_data:
                nlp.update(
                   [text],  # batch of texts
                   [annotations],  # batch of annotations
                   drop=0.2,  # dropout - make it harder to memorise data
                   sgd=optimizer,  # callable to update weights
                   losses=losses)

#             print(losses)
            
            print("Losses", losses)
    save_model(output_dir, nlp, 'st_ner')
    print(output_dir)

In [12]:
## creating data in spacy data input format

def get_training_data():
    train_data = []
    for index, row in train_df[:5000].iterrows():        
        selected_text = row.selected_text
        text = row.text
        start = text.find(selected_text)
        end = start + len(selected_text)
        train_data.append((text, {"entities": [(start, end, 'SKILL')]}))
    return train_data

In [13]:
def get_model_out_path():
    model_out_path = None
    model_out_path = '/models/model_skill'
    
    return model_out_path

In [22]:
train_data = get_training_data()
train_data[:1]

[('just ran thr contract negotiation ough the rain',
  {'entities': [(13, 33, 'SKILL')]})]

In [17]:
model_path = get_model_out_path()
model_path

'/models/model_skill'

In [66]:
import json
training_data = []
lines=[]

with open("SkillSet.json", encoding="utf8") as f:
    lines = f.readlines()
    #print (lines)
    
for line in lines:
    print("Inside Lines {line}")
    data = json.loads(line)
    text = data['content']
    entities = []
    for annotation in data['annotation']:
        #only a single point in text annotation.
        point = annotation['points'][0]
        labels = annotation['label']
        # handle both list of labels or a single label.
        if not isinstance(labels, list):
            labels = [labels]
            
        for label in labels:
           #dataturks indices are both inclusive [start, end] but spacy is not [start, end)
           entities.append((point['start'], point['end'] + 1 ,label))
           
        training_data.append((text, {"entities" : entities}))

Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}
Inside Lines {line}


# Delete this content which is mixing with main model

In [20]:
len(train_data)
# training_data[:10]

5000

In [25]:
from pathlib import Path
def train_SkillSet_NER(training_data,model=None, output_dir=model_path, n_iter=20):
                       
    """Load the model, set up the pipeline and train the entity recognizer."""
    if model is not None:
        nlp = spacy.load(model)  # load existing spaCy model
        print("Loaded model '%s'" % model)
    else:
        nlp = spacy.blank("en")  # create blank Language class
        print("Created blank 'en' model")

    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if "ner" not in nlp.pipe_names:
        ner = nlp.create_pipe("ner")
        nlp.add_pipe(ner, last=True)
    # otherwise, get it so we can add labels
    else:
        ner = nlp.get_pipe("ner")                  
     
    #print(nlp.pipe_names)

    if 'ner' not in nlp.pipe_names:
            ner = nlp.create_pipe('ner')
            nlp.add_pipe(ner, last=True)

    # add labels
    for _, annotations in training_data:
         for ent in annotations.get('entities'):
             ner.add_label(ent[2])

     # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']

    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()

        for itn in tqdm(range(n_iter)):
            print("Statring iteration " + str(itn))
            random.shuffle(training_data)
            losses = {}

            for text, annotations in training_data:
               nlp.update(
                   [text],  # batch of texts
                   [annotations],  # batch of annotations
                   drop=0.2,  # dropout - make it harder to memorise data
                   sgd=optimizer,  # callable to update weights
                   losses=losses)

            print("Losses", losses)                      
                       
                       
     # save model to output directory
    save_model(output_dir, nlp, 'st_ner')
    print(output_dir)

        # test the saved model

In [27]:
train_SkillSet_NER(train_data,output_dir='ModelData//SkillModel', model=None,n_iter=2)



  0%|                                                                                            | 0/2 [00:00<?, ?it/s]

Created blank 'en' model
Statring iteration 0




 50%|█████████████████████████████████████████▌                                         | 1/2 [05:31<05:31, 331.32s/it]

Losses {'ner': 5723.207422131477}
Statring iteration 1




100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [11:44<00:00, 352.49s/it]

Losses {'ner': 3399.180533825123}
Saved model to ModelData//SkillModel
ModelData//SkillModel


# End of Deletion for Data turk code

In [90]:
# for demo purpose i am just training the model for 2 iterations, feel free to experiment.
train(train_data[:1000], model_path, n_iter=20, model=None)

Created blank 'en' model


  5%|████▏                                                                              | 1/20 [00:11<03:30, 11.06s/it]

Losses {'ner': 244.53184608040425}


 10%|████████▎                                                                          | 2/20 [00:21<03:17, 10.98s/it]

Losses {'ner': 172.42652361771303}


...

Losses {'ner': 9.121594558452427}


 95%|█████████████████████████████████████████████████████████████████████████████▉    | 19/20 [03:37<00:08,  8.97s/it]

Losses {'ner': 1.407101637038274}


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [03:42<00:00, 11.14s/it]

Losses {'ner': 3.990713726843727}
Saved model to ModelData//SkillModel
/models/model_skill


In [31]:
def predict_entities(text, model):
    doc = model(text)
    ent_array = []
    for ent in doc.ents:
        start = text.find(ent.text)
        end = start + len(ent.text)
        new_int = [start, end, ent.label_]
        if new_int not in ent_array:
            ent_array.append([start, end, ent.label_])
    selected_text = text[ent_array[0][0]: ent_array[0][1]] if len(ent_array) > 0 else text
    return selected_text

In [29]:
import spacy
model_skill = spacy.load("ModelData\SkillModel")

In [32]:
predict_entities("Java, WCF, ASP.Net, C#, web services and many more".lower(), model_skill)

'web services'

In [33]:
desc = """
My computer is SO slooowww this morning.  I think it fine art photography `s a sign that I should go home and play in my yard.

Main Responsibility / Job Summary :

facial animation
autodesk motionbuilder
xsi
3d graphics
c++
low level programming
abap
microsoft office
microsoft excel
microsoft word
powerpoint
teaching
html
c++
c
ability to work under strict deadlines and pressure times
ability of multi-tasking
ability to lead multi-disciplined teams
strong leadership, project management and team building skills
strong problem solving and analytical skills
managing teams and work under critical deadlines and in challenging environments
strong business acumen
excellent communication skills
strong at business and peer relationship management



- Web based application development experience using Asp.Net, C#, JavaScript, Jquery, CSS, XML, AJAX and JSON.

"""
doc2 = model_skill(desc.lower())
print ("Entities= " + str(["" + str(ent.text) + " -> " + str(ent.label_) for ent in doc2.ents]))

Entities= ['photography -> SKILL', 'facial animation -> SKILL', 'microsoft office\nmicrosoft excel -> SKILL', 'leadership -> SKILL', 'xml -> SKILL']


In [35]:
model_dir = "ModelData\SkillModel"

nlp2 = spacy.load(model_dir)
desc = """
Java Developer with J23EE AND Tomcat"""
doc2 = nlp2(desc.lower())

print ("Entities= " + str(["" + str(ent.text) + " -> " + str(ent.label_) for ent in doc2.ents]))


Entities= []
